In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
from stardist import random_label_cmap
from stardist.models import StarDist3D
from skimage.transform import rescale
from skimage.measure import regionprops, regionprops_table
import pandas as pd
from pathlib import Path
import gc

np.random.seed(6)
lbl_cmap = random_label_cmap()

# Data
data_directory = Path('/media/el_tito/T7 Shield/20250605 histone compass/tiffed30z/h3k4me1/')
results_directory = Path('/media/el_tito/T7 Shield/20250605 histone compass/masks/h3k4me1/')

filelist = sorted(glob(os.path.join(data_directory, "*.tiff*")))


print(f"Found {len(filelist)} files.")

2025-09-19 14:40:37.437372: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-19 14:40:37.437404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-19 14:40:37.437411: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-19 14:40:37.485492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 321 files.


In [2]:
# Load model
demo_model = False
if demo_model:
    print(
        "NOTE: This is loading a previously trained demo model!\n"
        "      Please set the variable 'demo_model = False' to load your own trained model.",
        file=sys.stderr, flush=True
    )
    model = StarDist3D.from_pretrained('3D_demo')
else:
    model = StarDist3D(None, name='stardist', basedir='Fas3_round6')

# Process each image
for file_path in filelist:
    img = imread(file_path)
    
    # Determine number of channels
    n_channel = 1 if img.ndim == 3 else img.shape[-1]
    axis_norm = (0, 1, 2)

    if n_channel > 1:
        print(f"Normalizing image channels {'jointly' if axis_norm is None or 2 in axis_norm else 'independently'}")

    # Rearrange axes to (Z, Y, X, C)
    img = np.moveaxis(img, 1, 3)

    # Select and stack specific channels
    Fas3 = 1
    DAPI = 2
    Y = np.stack([
        img[:, :, :, Fas3],
        img[:, :, :, DAPI],
        img[:, :, :, DAPI]
    ], axis=3)

    # Downscale XY resolution
    Y = rescale(Y, scale=(1, 0.25, 0.25, 1), order=0, anti_aliasing=False)

    # Normalize and predict
    img_normalized = normalize(Y, 1, 99.8, axis=axis_norm)
    labels, details = model.predict_instances(img_normalized)

    # Save masks
    input_filename = Path(file_path).name
    output_filename = f"{input_filename}"
    output_path = results_directory / output_filename
    save_tiff_imagej_compatible(output_path, labels, axes='ZYX')

    # Cleanup
    del img, Y, img_normalized, labels, details
    gc.collect()

2025-09-19 14:41:08.768133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-19 14:41:08.865412: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-19 14:41:08.865661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Normalizing image channels jointly


2025-09-19 14:41:11.189599: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70090752 exceeds 10% of free system memory.
2025-09-19 14:41:11.272411: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70090752 exceeds 10% of free system memory.
2025-09-19 14:41:11.466668: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70090752 exceeds 10% of free system memory.
2025-09-19 14:41:11.636650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2025-09-19 14:41:14.581381: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-09-19 14:41:14.675126: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.19GiB with freed_by_coun

Normalizing image channels jointly


2025-09-19 14:41:26.788910: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70090752 exceeds 10% of free system memory.


Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image ch


Qhull output at end
qhull precision warning: 
The initial hull is narrow (cosine of min. angle is 0.9999999999999991).
Is the input lower dimensional (e.g., on a plane in 3-d)?  Qhull may
produce a wide facet.  Options 'QbB' (scale to unit box) or 'Qbb' (scale
last coordinate) may remove this warning.  Use 'Pp' to skip this warning.
See 'Limitations' in qh-impre.htm.


Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image channels jointly
Normalizing image ch